# KLUE BERT를 이용한 네이버 영화 리뷰 감성분석

런타임 유형을 **TPU**로 설정하세요.

원본 링크 : https://github.com/ukairia777/tensorflow-nlp-tutorial (<a href=https://creativecommons.org/licenses/by-nc-sa/2.0/kr/>CC BY-NC-SA 2.0 KR</a>)<br>
Modified by uramoon@kw.ac.kr

KLUE BERT는 한국어에 대해 학습된 BERT 모델입니다.
KLUE는 Korean Language Understanding Evaluation의 약어로 한국어 이해능력을 평가하는 벤치마크이며, BERT는 Bidirectional Encoder Representations from Transfomerers의 약어로 구글에서 개발한 자연어처리 모델입니다.


![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQc4oeBEEOISwWxlXG2X7XViY-2GAmHea9kUQ&usqp=CAU)

본 노트북에서는 KLUE BERT를 불러와서 전이학습으로 네이버 영화 리뷰에 대해 감성분석을 수행합니다.<br>
KLUE BERT는 모두의 말뭉치, 나무위키, 뉴스기사, 청와대 국민청원 등 62GB의 데이터로 훈련한 모델입니다.

## 패키지 설치
런타임 유형을 **TPU**로 설정하세요.

In [1]:
# KLUE BERT를사용하기 위한 패키지 설치
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
import transformers
from transformers import BertTokenizer, TFBertModel

## 데이터 준비하기

In [3]:
# 훈련 데이터와 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f4f05faef50>)

In [4]:
# TODO: DataFrame으로 읽기
train_data = pd.read_table('ratings_train.txt') #TODO
test_data = pd.read_table('ratings_test.txt') #TODO

In [5]:
# TODO: 데이터 크기 확인하기
print('훈련용 리뷰 개수 :', train_data.size) #TODO) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :', test_data.size) #TODO) # 테스트용 리뷰 개수 출력

훈련용 리뷰 개수 : 450000
테스트용 리뷰 개수 : 150000


In [6]:
# TODO: 훈련 데이터에서 처음 5개 출력하기
train_data[0:5] #TODO] # 상위 5개 출력

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
# TODO: 테스트 데이터에서 처음 5개 출력하기
test_data[0:5] #TODO] # 상위 5개 출력

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [8]:
# 훈련 데이터에서 NULL 값을 포함한 행 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.reset_index(drop=True) # 인덱스 재설정
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [9]:
# TODO: 테스트 데이터에서 NULL 값을 포함한 행 제거
test_data = test_data.dropna(how = 'any') #TODO: Null 값이 존재하는 행 제거
test_data = test_data.reset_index(drop=True) #TODO: 인덱스 재설정
print(test_data.isnull().values.any()) #TODO: Null 값이 존재하는지 확인)

False


In [10]:
# TODO: NULL 제거 후 데이터 크기 확인하기
print('훈련용 리뷰 개수 :', train_data.size) #TODO) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :', test_data.size) #TODO) # 테스트용 리뷰 개수 출력

훈련용 리뷰 개수 : 449985
테스트용 리뷰 개수 : 149991


## 토큰화

지난 노트북과 달리 토크나이저를 직접 만들지 않고 이미 훈련이 완료된 모델에서 불러옵니다.

In [11]:
# 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [12]:
# 단어를 정수로 인코딩
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [13]:
# 문자열을 토큰화, ##은 해당 토큰이 어절의 시작이 아님을 알려줍니다.
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '##내', '##내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [14]:
# 문자열을 인코딩 (인공신경망이 이해하는 형태로 변환) 후 디코딩하기 (사람이 이해하는 형태로 변환)
# [CLS]는 BERT에서 문서의 시작을 나타내는 토큰
# [SEP]는 문장의 끝을 나타내는 토큰
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [15]:
# 토큰을 하나씩 출력하는 코드
for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

[ C L S ]
보
# # 는
# # 내
# # 내
그 대 로
들 어 맞
# # 는
예 측
카 리 스 마
없
# # 는
악 역
[ S E P ]


In [16]:
# TODO: 다음 문자열을 토큰화 하세요.
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화")) #TODO("전율을 일으키는 영화. 다시 보고싶은 영화"))

['전', '##율', '##을', '일으키', '##는', '영화', '.', '다시', '보고', '##싶', '##은', '영화']


In [17]:
# TODO: 다음 문자열을 정수로 인코딩 하세요.
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화")) #TODO("전율을 일으키는 영화. 다시 보고싶은 영화"))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3]


각 리뷰에서 몇 개의 토큰을 사용할 것인지 설정합니다. 지난 노트북과 동일하게 하기 위해 36으로 설정합니다.

In [18]:
# 모든 리뷰 토큰 개수의 평균 + 3 * 표준편차 = 36
# 평균 + 2 * 표준편차를 사용하신 분들도 있습니다.
max_seq_len = 36

In [19]:
# 36개의 토큰으로 문장 인코딩하기, 길이가 짧기 때문에 남는 자리는 0으로 채워집니다. (padding)
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, padding='max_length', truncation=True)
print(encoded_result)
print('길이 :', len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이 : 36


## 데이터셋 만들기

훈련 데이터셋 (X_train: 리뷰, y_train: 레이블 [0:부정, 1:긍정])과 테스트 데이터셋 (X_test, y_test)를 만듭니다.

In [20]:
# 훈련 데이터셋 만들기

X_train = []

for sentence in tqdm(train_data['document']): # document에서 리뷰 하나씩 가져오기, tqdm이 진행률을 보여줍니다.
  encoded_sentence = tokenizer.encode(sentence, max_length=max_seq_len, padding='max_length', truncation=True)
  X_train.append(encoded_sentence)  

X_train = np.array(X_train, dtype=int)
y_train = np.array(train_data['label'])

100%|██████████| 149995/149995 [00:37<00:00, 4025.43it/s]


In [22]:
# TODO: 테스트 데이터셋 만들기

X_test = []

# TODO
for sentence in tqdm(test_data['document']):
    encoded_sentence = tokenizer.encode(sentence, max_length=max_seq_len, padding='max_length', truncation=True)
    X_test.append(encoded_sentence)

X_test = np.array(X_test, dtype=int) #TODO
y_test = np.array(test_data['label']) #TODO

100%|██████████| 49997/49997 [00:12<00:00, 3940.80it/s]


## TPU 이용하기
TPU 사용법 : https://wikidocs.net/119990

In [23]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드, 이해하실 필요는 없습니다.
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

## 모델 생성

지금까지는 이전 층의 출력이 다음 층에 모두 연결되는 Sequential 모델만을 사용했습니다. 우리는 BERT가 내보내는 출력 중 일부만을 사용하기 때문에 Sequential 모델을 사용하지 않고, Functional API를 사용해 일반 함수와 같이 각 층의 입력과 출력을 지정하여 모델을 만들 것입니다.

In [24]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

with strategy.scope(): # TPU 이용하기
  # 입력층
  inputs = Input(shape=(max_seq_len,), dtype=tf.int32) # max_seq_len (36) 개의 토큰을 입력받아 inputs로 출력합니다.

  # KLUE BERT 모델 불러오기
  bert = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)(inputs) # inputs을 입력으로 받아 bert로 출력합니다.

  # 출력층
  output = Dense(1, activation='sigmoid')(bert[1]) # bert의 출력 중 두 번째 원소를 입력받아 하나로 선형결합하여 sigmoid로 0 혹은 1을 output에 출력합니다.

  # 모델 정의
  model = Model(inputs=inputs, outputs=output) # 모델의 입력은 inputs, 출력은 output

  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy']) 

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [25]:
# 시간이 오래 걸리니 epochs을 2로 설정하여 훈련합니다.
model.fit(X_train, y_train, epochs=2, batch_size=64, validation_split=0.2)

Epoch 1/2
1875/1875 [==============================] - 220s 79ms/step - loss: 0.3013 - accuracy: 0.8692 - val_loss: 0.2487 - val_accuracy: 0.8983
Epoch 2/2
1875/1875 [==============================] - 135s 72ms/step - loss: 0.2034 - accuracy: 0.9190 - val_loss: 0.2504 - val_accuracy: 0.8999


In [26]:
# 테스트 데이터셋으로 평가합니다.
results = model.evaluate(X_test, y_test, batch_size=1024)
print("test loss, test acc: ", results)

49/49 [==============================] - 15s 197ms/step - loss: 0.2578 - accuracy: 0.8972
test loss, test acc:  [0.25784364342689514, 0.8972338438034058]


<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FW0TLL%2FbtqRx0uGeC2%2FKxbOgpwhzXXvwu1VtcmjNK%2Fimg.jpg' height=200>

직접 모델을 만든 노트북에서는 85%를 넘는 것이 쉽지는 않았습니다. <br>
2023년 1학기 기준으로 한정된 컴퓨팅 자원에서 더 좋은 성능을 원한다면 polyglot (한국어), LLaMA, Alpaca 등을 이용해 보세요.<br> (PyTorch를 사용하는 것이 유리할 수 있습니다.)

In [28]:
# TODO: model을 이용하여 주어진 문장을 분류하는 함수를 만들어 보세요.

def sentiment_predict(new_sentence):
  encoded_input = tokenizer.encode(new_sentence, max_length=max_seq_len, padding='max_length', truncation=True) #TODO
  encoded_input = np.array([encoded_input])
  score = model.predict(encoded_input)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [29]:
sentiment_predict('보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에 ')

1/1 [==============================] - 8s 8s/step
97.05% 확률로 부정 리뷰입니다.



In [30]:
sentiment_predict("스토리는 확실히 실망이였지만 배우들 연기력이 대박이였다 특히 이제훈 연기 정말 ... 이 배우들로 이렇게밖에 만들지 못한 영화는 아쉽지만 배우들 연기력과 사운드는 정말 빛났던 영화. 기대하고 극장에서 보면 많이 실망했겠지만 평점보고 기대없이 집에서 편하게 보면 괜찮아요. 이제훈님 연기력은 최고인 것 같습니다")

1/1 [==============================] - 1s 503ms/step
72.50% 확률로 긍정 리뷰입니다.



In [31]:
sentiment_predict("남친이 이 영화를 보고 헤어지자고한 영화. 자유롭게 살고 싶다고 한다. 내가 무슨 나비를 잡은 덫마냥 나에겐 다시 보고싶지 않은 영화.")

1/1 [==============================] - 1s 518ms/step
70.67% 확률로 부정 리뷰입니다.



In [32]:
sentiment_predict("이 영화 존잼입니다 대박")

1/1 [==============================] - 1s 559ms/step
94.49% 확률로 긍정 리뷰입니다.



In [33]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

1/1 [==============================] - 1s 572ms/step
92.17% 확률로 긍정 리뷰입니다.



In [34]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

1/1 [==============================] - 1s 521ms/step
98.92% 확률로 부정 리뷰입니다.



In [35]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

1/1 [==============================] - 1s 515ms/step
98.95% 확률로 부정 리뷰입니다.



In [36]:
sentiment_predict('감독 뭐하는 놈이냐?')

1/1 [==============================] - 1s 504ms/step
99.15% 확률로 부정 리뷰입니다.



In [37]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

1/1 [==============================] - 1s 540ms/step
86.70% 확률로 긍정 리뷰입니다.

